In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [28]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print(len(contents))
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

1875


In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
from random import randint as r

In [31]:
import time

In [32]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=14)
        self.fc2 = nn.Linear(in_features=14, out_features=4)
        #self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)

        #t = F.softmax(t)
        
        return t

In [ ]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr=0.0000000001)
lossDuplicationCounter = 0
prevloss=0
loss = 4
i=0

In [ ]:
start=time.time()
while ((loss > 0.0005) and (lossDuplicationCounter<10)):
    a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(a*15):(a*15+14)])
    optimizer.zero_grad()   # zero the gradient buffers
    out = net(chars)
    if (contents[a*14+15]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[a*14+15]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[a*14+15]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    elif (contents[a*14+15]=='Artur'):
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    if prevloss == loss:
        lossDuplicationCounter +=1
        print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    loss.backward(retain_graph=True)
    #loss.backward()
    optimizer.step()   # Does the update
    if i%1000==0:
        print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))
    i+=1
print("Learning time: \n\t", time.time()-start)
torch.cuda.empty_cache()



iteration:  0 
Loss:  20805842. 
Output:  [-6805.1733, -5400.6064, -1745.0739, -2167.2136],
 
Target:  [0., 0., 0., 1.]


iteration:  1000 
Loss:  35397.9609 
Output:  [  51.9023, -136.5978, -219.7746,  267.7013] 
Target:  [0., 1., 0., 0.]


iteration:  2000 
Loss:  1228.0507 
Output:  [ 12.6701, -58.0985, -22.1364,  28.9991] 
Target:  [0., 0., 1., 0.]


iteration:  3000 
Loss:  4960.6523 
Output:  [ 15.2907, -76.6994, -54.3176, 103.0592] 
Target:  [0., 1., 0., 0.]


iteration:  4000 
Loss:  9911.8164 
Output:  [ 28.8062, -90.9083, -92.9538, 147.3969] 
Target:  [0., 0., 1., 0.]


iteration:  5000 
Loss:  396041.1250 
Output:  [ -92.2964,  511.3264,  618.6121, -965.7880] 
Target:  [0., 0., 1., 0.]


iteration:  6000 
Loss:  772.0061 
Output:  [  3.1337, -29.2497, -28.0656,  38.8811] 
Target:  [0., 0., 0., 1.]


iteration:  7000 
Loss:  563.7755 
Output:  [  5.8037, -33.2869, -21.4543,  24.6820] 
Target:  [0., 0., 1., 0.]


iteration:  8000 
Loss:  49135.8281 
Output:  [  61.8116, -212

In [ ]:
print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))

In [ ]:
print
i=0
for filename in glob.glob(path):
    #a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(i*15):(i*15+14)])
    #print(str(net(chars)).strip(", grad_fn=<AddBackward0>)").strip("tensor("), "\t : ",contents[i*15+14])
    print(((str(net(chars))).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'), " : ", contents[i*15+14])
    i=i+1

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(net, "net.pt")


In [ ]:
haba = torch.load("net.pt")
haba.eval()


In [ ]:


i=0
ilezg=0
for filename in glob.glob(path):
    tt=torch.tensor(contents[(a*15):(a*15+14)])
    who = contents[i*15+14]
    predykcja = haba(tt)
    predlist = predykcja.tolist().index(max(predykcja.tolist()))
    predykcjaOsoba=''
    if predlist == 0:
        predykcjaOsoba='Marcin'
    elif predlist == 1:
        predykcjaOsoba='Bartek'
    elif predlist == 2:
        predykcjaOsoba='Maciek'
    elif predlist == 3:
        predykcjaOsoba='Artur'
    else:
        print('ERROR!!!')
    print(i, who, ' : \n', str(tt).strip('tensor(').strip(')'), '\n', str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)'))
    i+=1
    if who == predykcjaOsoba:
        ilezg+=1
print(ilezg/i*100)
